1- **Twitter EXERCISE:** 

Given the starting user "gatdelapena", retrieve the user name, the date of the account creation and the last tweet date and text for each follower. Store them to a database called 'twitter' and into a collection called 'info'. The attributes of the document are: 'screen_name', 'created_at', 'last_tweet_text' and 'last_tweet_created_at'. 


In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import json
import pymongo
import tweepy

with open('consumer_key.txt', 'r') as f:
    consumer_key =  f.read()
f.closed

with open('consumer_secret.txt', 'r') as f:
    consumer_secret = f.read()
f.closed

with open('access_key.txt', 'r') as f:
    access_key = f.read()
f.closed

with open('access_secret.txt', 'r') as f:
     access_secret = f.read()
f.closed


#Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)

True

True

True

True

In [3]:
USER_NAME = "gatdelapena"

# Retrieve the followers
import time
ids = []
for page in tweepy.Cursor(api.followers_ids, screen_name=USER_NAME).pages():
    ids.extend(page)
    time.sleep(5)

# Create the connection to MongoDB
try:
    with open("credentials.txt", 'r') as f:
        [name,password,url]=f.read().splitlines()
    conn=pymongo.MongoClient("mongodb+srv://{}:{}@{}".format(name,password,url))
    print ("Connected successfully!!!")
except pymongo.errors.ConnectionFailure as e:
    print ("Could not connect to MongoDB: %s" % e) 
db = conn['twitter']
collection = db['info']

# Clear already existing files in the collection
collection.delete_many({})

# Create a document storing all the data
for usr in ids:
    document={}
    user = api.get_user(id=usr)
    document['screen_name'] = user.screen_name
    document['created_at'] = user.created_at
    try:
        last_tweet = api.user_timeline(id=usr,count=1)
        for status in last_tweet:
            document['last_tweet_text'] = status.text
            document['last_tweet_created_at'] = status.created_at
    except tweepy.TweepError:
        print("Not authorized")
    collection.insert_one(document)

print("Done!!!")

Connected successfully!!!


Not authorized


Not authorized


Not authorized


Done!!!


2- **Selenium EXERCISE:** 
<ul>
<li> Open a browser 

<li>Go to tripadvisor/Restaurants

<li>Find the search text box

<li>Clear it, input the query "Sant Cugat" and send it

<li>Go to "Restaurants" and get the name and link of the top 10 restaurants in Sant cugat
<li> Store them into a database called 'TripAdvisor' and a collection called 'Restaurants' with the documents atributes 'name' and 'link'
</ul>

</div>

In [4]:
%reset -f
#!/usr/bin/python
# -*- coding: utf-8 -*-

from selenium import webdriver
import time,os


url = 'https://www.tripadvisor.es/Restaurants'
path = "D:/Dokumenty/ESADE/Cloud_Computing/Session4_5/chromedriver.exe"
browser = webdriver.Chrome(path)

# Open TripAdvisor
browser.get(url)

# Clear the search text box and insert Sant Cugat
element = browser.find_element_by_xpath('//*[@id="taplc_trip_search_home_restaurants_0"]/div[2]/div[1]/div/span/input')
element.clear()
s='Sant Cugat'
element.send_keys(s)
browser.implicitly_wait(5)

# Search the results
browser.find_element_by_xpath('//*[@id="SUBMIT_RESTAURANTS"]/span[2]').click()
browser.implicitly_wait(5)

In [5]:
# Create the connection to MongoDB
import pymongo

try:
    with open("credentials.txt", 'r') as f:
        [name,password,url]=f.read().splitlines()
    conn=pymongo.MongoClient("mongodb+srv://{}:{}@{}".format(name,password,url))
    print ("Connected successfully!!!")
except pymongo.errors.ConnectionFailure as e:
    print ("Could not connect to MongoDB: %s" % e) 
db = conn['TripAdvisor']
collection = db['Restaurants']

# Clear already existing files in the collection
collection.delete_many({})

Connected successfully!!!


In [6]:
# Find all elements with restaurants data
rest = browser.find_elements_by_css_selector('a.restaurants-list-ListCell__restaurantName--2aSdo')

In [7]:
# Filter restaurants to find only top10 and insert data into document
for i in range(len(rest)):
    try:
        if int(''.join(filter(str.isdigit, rest[i].text[:4]))) in list(range(1,11,1)):
            document = {}
            document['name'] = rest[i].text
            document['link'] = rest[i].get_attribute('href')
            collection.insert_one(document)
    except:
        continue

browser.quit()
print("Done!!!")

Done!!!
